### Connect to DB

In [3]:
import psycopg2, os
import pandas as pd

In [221]:
import psycopg2, os

print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="abc_foodmart",
    user="postgres",
    password="jade5808")

Connecting to the PostgreSQL database...


In [222]:
# create a cursor
cur = conn.cursor()

### Create SQL Tables for ABC Foodmart

In [ ]:
createCmd = """ 
-- Department table
CREATE TABLE department (
    dept_id SERIAL PRIMARY KEY,
    dept_name VARCHAR(100) NOT NULL
);

-- Positions table
CREATE TABLE position (
    position_id SERIAL PRIMARY KEY,
    position_name VARCHAR(100) NOT NULL
);

-- Store table
CREATE TABLE store (
    store_id SERIAL PRIMARY KEY,
    store_name VARCHAR(100),
    state VARCHAR(20),
    city VARCHAR(50),
    street VARCHAR(50),
    zip VARCHAR(10)
);

-- Employee table
CREATE TABLE employee (
    employee_id SERIAL PRIMARY KEY,
    store_id INT NOT NULL,
    dept_id INT NOT NULL,
    position_id INT NOT NULL,
    first_name VARCHAR(30) NOT NULL,
    last_name VARCHAR(30) NOT NULL,
    email VARCHAR(100) NOT NULL,
    phone_number VARCHAR(15) NOT NULL,
    state VARCHAR(20) NOT NULL,
    city VARCHAR(50) NOT NULL,
    street VARCHAR(100) NOT NULL,
    zip VARCHAR(10) NOT NULL,
    hire_date DATE NOT NULL,
    CONSTRAINT fk_store FOREIGN KEY (store_id) REFERENCES store(store_id),
    CONSTRAINT fk_dept FOREIGN KEY (dept_id) REFERENCES department(dept_id),
    CONSTRAINT fk_position FOREIGN KEY (position_id) REFERENCES position(position_id)
);

-- Payroll table
CREATE TABLE payroll (
    payroll_id SERIAL PRIMARY KEY,
    employee_id INT NOT NULL,
    pay_period_start DATE NOT NULL,
    pay_period_end DATE NOT NULL,
    hour_wage NUMERIC(10,2) NOT NULL,
    CONSTRAINT fk_employee_payroll FOREIGN KEY (employee_id) REFERENCES employee(employee_id)
);

-- Employee Schedule table
CREATE TABLE employee_schedule (
    schedule_id SERIAL PRIMARY KEY,
    employee_id INT NOT NULL,
    start_time TIMESTAMP,
    end_time TIMESTAMP,
    CONSTRAINT fk_employee_schedule FOREIGN KEY (employee_id) REFERENCES employee(employee_id) 
        ON DELETE CASCADE ON UPDATE CASCADE
);

-- Employee Time Tracking table
CREATE TABLE employee_time_tracking (
    time_tracking_id SERIAL PRIMARY KEY,
    employee_id INT NOT NULL,
    start_time TIMESTAMP NOT NULL,
    end_time TIMESTAMP,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_employee_time_tracking FOREIGN KEY (employee_id) REFERENCES employee(employee_id),
    CONSTRAINT fk_schedule_time_tracking FOREIGN KEY (schedule_id) REFERENCES employee_schedule(schedule_id),
    CONSTRAINT chk_end_after_start CHECK (end_time IS NULL OR end_time > start_time)
);

-- Vendors table
CREATE TABLE vendors (
    vendor_id SERIAL PRIMARY KEY,
    vendor_name VARCHAR(20) UNIQUE NOT NULL,
    phone_number VARCHAR(12),
    email VARCHAR(50),
    website VARCHAR(100),
    CONSTRAINT chk_vendor_contact CHECK (
        (phone_number IS NOT NULL OR email IS NOT NULL OR website IS NOT NULL)
    )
);

-- Products table
CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    dept_id INT,
    vendor_id INT,
    brand VARCHAR(50),
    product_name VARCHAR(100),
    CONSTRAINT fk_dept_products FOREIGN KEY (dept_id) REFERENCES department(dept_id),
    CONSTRAINT fk_vendor_products FOREIGN KEY (vendor_id) REFERENCES vendors(vendor_id)
);

-- Pricing History table
CREATE TABLE price_history (
    history_id SERIAL PRIMARY KEY,
    product_id INT NOT NULL,
    store_id INT,
    price DECIMAL(10, 2) NOT NULL CHECK (price > 0.01),
    effective_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_product_price_history FOREIGN KEY (product_id) REFERENCES products(product_id),
    CONSTRAINT fk_store_price_history FOREIGN KEY (store_id) REFERENCES store(store_id)
);

-- Deliveries table
CREATE TABLE deliveries (
    delivery_id SERIAL PRIMARY KEY,
    product_id INT NOT NULL,
    vendor_id INT NOT NULL,
    store_id INT NOT NULL,
    delivery_date DATE NOT NULL,
    quantity INT NOT NULL,
    CONSTRAINT fk_product_deliveries FOREIGN KEY (product_id) REFERENCES products(product_id),
    CONSTRAINT fk_vendor_deliveries FOREIGN KEY (vendor_id) REFERENCES vendors(vendor_id),
    CONSTRAINT fk_store_deliveries FOREIGN KEY (store_id) REFERENCES store(store_id),
    CONSTRAINT chk_delivery_quantity CHECK (quantity > 0)
);

-- Inventory table
CREATE TABLE inventory (
    inventory_id SERIAL PRIMARY KEY,
    store_id INT NOT NULL,
    product_id INT NOT NULL,
    quantity INT NOT NULL DEFAULT 0,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_store_inventory FOREIGN KEY (store_id) REFERENCES store(store_id),
    CONSTRAINT fk_product_inventory FOREIGN KEY (product_id) REFERENCES products(product_id),
    CONSTRAINT chk_inventory_quantity CHECK (quantity >= 0)
);

-- Transactions table
CREATE TABLE transactions (
    transaction_id SERIAL PRIMARY KEY,
    store_id INT NOT NULL,
    transaction_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    transaction_type INT NOT NULL CHECK (transaction_type IN (0, 1)), -- 0 is for sale and 1 is for a return
    CONSTRAINT fk_store_transactions FOREIGN KEY (store_id) REFERENCES store(store_id)
);

-- Sales table
CREATE TABLE sales (
    sale_id SERIAL PRIMARY KEY,
    transaction_id INT NOT NULL,
    product_id INT NOT NULL,
    quantity INT NOT NULL,
    CONSTRAINT fk_transaction_sales FOREIGN KEY (transaction_id) REFERENCES transactions(transaction_id),
    CONSTRAINT fk_product_sales FOREIGN KEY (product_id) REFERENCES products(product_id),
    CONSTRAINT chk_sale_quantity CHECK (quantity > 0)
);

-- Customers table
CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    phone_number VARCHAR(15),
    date_joined TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Customer Reviews table
CREATE TABLE customer_reviews (
    review_id SERIAL PRIMARY KEY,
    customer_id INT NOT NULL,
    store_id INT,
    product_id INT,
    rating INT NOT NULL CHECK (rating >= 1 AND rating <= 5),
    review_text TEXT,
    review_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_customer_reviews FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    CONSTRAINT fk_store_reviews FOREIGN KEY (store_id) REFERENCES store(store_id),
    CONSTRAINT fk_product_reviews FOREIGN KEY (product_id) REFERENCES products(product_id),
    CONSTRAINT chk_review_entity CHECK (store_id IS NOT NULL OR product_id IS NOT NULL)
);

-- Product Waste table
CREATE TABLE product_waste (
    waste_id SERIAL PRIMARY KEY,
    product_id INT NOT NULL,
    store_id INT NOT NULL,
    dept_id INT,
    quantity INT NOT NULL CHECK (quantity > 0),
    waste_reason VARCHAR(255) NOT NULL,
    waste_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT fk_product_waste FOREIGN KEY (product_id) REFERENCES products(product_id),
    CONSTRAINT fk_store_waste FOREIGN KEY (store_id) REFERENCES store(store_id),
    CONSTRAINT fk_dept_waste FOREIGN KEY (dept_id) REFERENCES department(dept_id)
);
            """

cur.execute(createCmd)
conn.commit()

### Trigger Statements

In [ ]:
createCmd = """ 
-- Add a trigger to handle updating the `updated_at` column
CREATE OR REPLACE FUNCTION update_timestamp()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = CURRENT_TIMESTAMP;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER set_updated_at
BEFORE UPDATE ON employee_time_tracking
FOR EACH ROW
EXECUTE FUNCTION update_timestamp();


CREATE OR REPLACE FUNCTION update_inventory_on_delivery()
RETURNS TRIGGER AS $$
BEGIN
    -- Check if the inventory record already exists
    IF EXISTS (
        SELECT 1
        FROM inventory
        WHERE store_id = NEW.store_id AND product_id = NEW.product_id
    ) THEN
        -- Update the existing inventory record
        UPDATE inventory
        SET quantity = quantity + NEW.quantity,
            last_updated = CURRENT_TIMESTAMP
        WHERE store_id = NEW.store_id AND product_id = NEW.product_id;
    ELSE
        -- Insert a new inventory record
        INSERT INTO inventory (store_id, product_id, quantity, last_updated)
        VALUES (NEW.store_id, NEW.product_id, NEW.quantity, CURRENT_TIMESTAMP);
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trigger_update_inventory
AFTER INSERT ON deliveries
FOR EACH ROW
EXECUTE FUNCTION update_inventory_on_delivery();


CREATE OR REPLACE FUNCTION update_inventory_on_sales()
RETURNS TRIGGER AS $$
BEGIN
    -- Check if the inventory record exists
    IF EXISTS (
        SELECT 1
        FROM inventory
        WHERE store_id = (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id)
          AND product_id = NEW.product_id
    ) THEN
        -- If it's a sale, decrease inventory quantity
        IF (SELECT transaction_type FROM transactions WHERE transaction_id = NEW.transaction_id) = 0 THEN
            UPDATE inventory
            SET quantity = quantity - NEW.quantity,
                last_updated = CURRENT_TIMESTAMP
            WHERE store_id = (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id)
              AND product_id = NEW.product_id;
        -- If it's a return, increase inventory quantity
        ELSE
            UPDATE inventory
            SET quantity = quantity + NEW.quantity,
                last_updated = CURRENT_TIMESTAMP
            WHERE store_id = (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id)
              AND product_id = NEW.product_id;
        END IF;
    ELSE
        -- If no inventory record exists, handle gracefully (optional)
        RAISE EXCEPTION 'Inventory record not found for store_id % and product_id %',
            (SELECT store_id FROM transactions WHERE transaction_id = NEW.transaction_id),
            NEW.product_id;
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trigger_update_inventory_sales
AFTER INSERT ON sales
FOR EACH ROW
EXECUTE FUNCTION update_inventory_on_sales();

            """

cur.execute(createCmd)
conn.commit()

### Script to Create Data

#### Department Data

In [5]:
import pandas as pd

def create_department_list():
    """
    Creates a list of grocery store departments with their IDs and names
    and returns it as a Pandas DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing department IDs and names.
    """
    # Example department data
    departments = [
        {"dept_id": 1, "dept_name": "Produce"},
        {"dept_id": 2, "dept_name": "Dairy"},
        {"dept_id": 3, "dept_name": "Bakery"},
        {"dept_id": 4, "dept_name": "Frozen Foods"},
        {"dept_id": 5, "dept_name": "Meat & Seafood"},
        {"dept_id": 6, "dept_name": "Pantry"},
        {"dept_id": 7, "dept_name": "Beverages"},
        {"dept_id": 8, "dept_name": "Household Supplies"},
        {"dept_id": 9, "dept_name": "Health & Beauty"},
        {"dept_id": 10, "dept_name": "Snacks"}
    ]

    # Create DataFrame
    df = pd.DataFrame(departments)
    
    return df

# Usage
department_df = create_department_list()
department_df

,dept_id,dept_name
0,1,Produce
1,2,Dairy
2,3,Bakery
3,4,Frozen Foods
4,5,Meat & Seafood
5,6,Pantry
6,7,Beverages
7,8,Household Supplies
8,9,Health & Beauty
9,10,Snacks


In [6]:
# Define the table and column structure
table_name = "department"
columns = ", ".join(department_df.columns)
placeholders = ", ".join(["%s"] * len(department_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in department_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the department table.


#### Positions Data

In [7]:
import pandas as pd

# Function to generate position data and return a DataFrame
def generate_position_data(num_records=10):
    """
    Generates position data and returns it as a DataFrame.
    
    Args:
        num_records (int): Number of records to generate.

    Returns:
        pd.DataFrame: A DataFrame containing position data.
    """
    # List of typical food store job positions
    position_names = [
        "Cashier", "Stock Clerk", "Store Manager", "Assistant Manager", 
        "Customer Service Representative", "Bakery Staff", "Deli Clerk", 
        "Meat Cutter", "Produce Clerk", "Sales Associate", "Grocery Bagger", 
        "Inventory Manager"
    ]
    
    # Generate data for the DataFrame
    data = []
    for position_id in range(1, num_records + 1):
        position_name = position_names[position_id % len(position_names)]  # Loop over position names
        data.append({"position_id": position_id, "position_name": position_name})
    
    # Create a DataFrame
    df = pd.DataFrame(data)
    return df

# Example usage: generate 12 position records
position_df = generate_position_data(12)

# Display the DataFrame
print(position_df)

    position_id                    position_name
0             1                      Stock Clerk
1             2                    Store Manager
2             3                Assistant Manager
3             4  Customer Service Representative
4             5                     Bakery Staff
5             6                       Deli Clerk
6             7                      Meat Cutter
7             8                    Produce Clerk
8             9                  Sales Associate
9            10                   Grocery Bagger
10           11                Inventory Manager
11           12                          Cashier


In [8]:
# Define the table and column structure
table_name = "position"
columns = ", ".join(position_df.columns)
placeholders = ", ".join(["%s"] * len(position_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in position_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the position table.


#### Stores Data 

In [9]:
import pandas as pd

# Function to create a DataFrame for store data
def generate_store_data():
    """
    Generates store data for three stores located in Queens and Brooklyn 
    and returns it as a DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing store data.
    """
    # Store data for three specific stores
    stores = [
        {
            "store_id": 1,
            "store_name": "ABC Food Mart Queens 1",
            "state": "NY",
            "city": "Queens",
            "street": "Main Street",
            "zip": "11354"
        },
        {
            "store_id": 2,
            "store_name": "ABC Food Mart Queens 2",
            "state": "NY",
            "city": "Queens",
            "street": "Broadway",
            "zip": "11372"
        },
        {
            "store_id": 3,
            "store_name": "ABC Food Mart Brooklyn",
            "state": "NY",
            "city": "Brooklyn",
            "street": "Flatbush Avenue",
            "zip": "11226"
        }
    ]
    
    # Create a DataFrame
    df = pd.DataFrame(stores)
    return df

# Example usage
store_df = generate_store_data()

# Display the DataFrame
print(store_df)


   store_id              store_name state      city           street    zip
0         1  ABC Food Mart Queens 1    NY    Queens      Main Street  11354
1         2  ABC Food Mart Queens 2    NY    Queens         Broadway  11372
2         3  ABC Food Mart Brooklyn    NY  Brooklyn  Flatbush Avenue  11226


In [10]:
# Define the table and column structure
table_name = "store"
columns = ", ".join(store_df.columns)
placeholders = ", ".join(["%s"] * len(store_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in store_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the store table.


#### Employee Data

In [28]:
import pandas as pd
import random
from faker import Faker

# Initialize Faker
fake = Faker()

def generate_phone_number():
    """
    Generates a phone number with exactly 12 characters in the format 'XXX-XXX-XXXX'.
    Returns:
        str: A formatted phone number string.
    """
    return f"{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"

# Function to generate employee data
def generate_employee_data(num_employees_per_store=30, stores=None, departments=None, positions=None):
    """
    Generates employee data for multiple stores.

    Args:
        num_employees_per_store (int): Number of employees to generate for each store.
        stores (list): List of store data dictionaries.
        departments (list): List of department data dictionaries.
        positions (list): List of position data dictionaries.

    Returns:
        pd.DataFrame: A DataFrame containing employee data.
    """
    if stores is None or departments is None or positions is None:
        raise ValueError("Stores, departments, and positions data must be provided.")

    employee_data = []
    employee_id = 1  # Start employee ID

    for store in stores:
        for _ in range(num_employees_per_store):
            dept = random.choice(departments)
            position = random.choice(positions)
            employee = {
                "employee_id": employee_id,
                "store_id": store["store_id"],
                "dept_id": dept["dept_id"],
                "position_id": position["position_id"],
                "first_name": fake.first_name(),
                "last_name": fake.last_name(),
                "email": fake.email(),
                "phone_number": generate_phone_number(),
                "state": store["state"],
                "city": store["city"],
                "street": fake.street_name(),
                "zip": store["zip"],
                "hire_date": fake.date_between(start_date="-5y", end_date="today"),
            }
            employee_data.append(employee)
            employee_id += 1

    return pd.DataFrame(employee_data)

# Example store, department, and position data
stores = [
    {"store_id": 1, "state": "NY", "city": "Queens", "street": "Main Street", "zip": "11354"},
    {"store_id": 2, "state": "NY", "city": "Queens", "street": "Broadway", "zip": "11372"},
    {"store_id": 3, "state": "NY", "city": "Brooklyn", "street": "Flatbush Avenue", "zip": "11226"},
]

departments = [{"dept_id": i, "dept_name": f"Department {i}"} for i in range(1, 11)]  # 10 Departments
positions = [{"position_id": i, "position_name": f"Position {i}"} for i in range(1, 13)]  # 12 Positions

# Generate employee data
employee_df = generate_employee_data(
    num_employees_per_store=30,
    stores=stores,
    departments=departments,
    positions=positions,
)


In [29]:
# Display the DataFrame
employee_df

,employee_id,store_id,dept_id,position_id,first_name,last_name,email,phone_number,state,city,street,zip,hire_date
0,1,1,1,5,Robert,Vaughn,kevinwilliams@example.net,874-143-9623,NY,Queens,Parker Isle,11354,2021-10-21
1,2,1,1,8,Michelle,Taylor,crystal43@example.net,875-688-2583,NY,Queens,Sarah Dam,11354,2023-12-29
2,3,1,8,1,Charles,Olson,melissa72@example.net,604-955-5681,NY,Queens,Savannah Course,11354,2020-03-07
3,4,1,9,1,Jeremy,Strong,jthomas@example.org,307-381-9382,NY,Queens,Adams Loop,11354,2022-11-10
4,5,1,6,2,Penny,Johnson,leecristina@example.org,323-725-7944,NY,Queens,Daniel Drive,11354,2020-11-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,86,3,3,9,Jenny,Malone,laura04@example.net,483-476-9377,NY,Brooklyn,Green Heights,11226,2023-12-30
86,87,3,8,3,Matthew,Cummings,timothyparsons@example.com,968-334-6138,NY,Brooklyn,Jessica Track,11226,2023-08-05
87,88,3,2,8,Trevor,Valdez,banksnancy@example.com,779-309-3829,NY,Brooklyn,Morgan Forges,11226,2023-01-21
88,89,3,10,5,Derek,Koch,cwhite@example.org,652-209-2493,NY,Brooklyn,Sandra Ports,11226,2021-01-11


In [33]:
# Define the table and column structure
table_name = "employee"
columns = ", ".join(employee_df.columns)
placeholders = ", ".join(["%s"] * len(employee_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in employee_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the employee table.


#### employee schedule

In [34]:
import pandas as pd
import random
from datetime import datetime, timedelta
from faker import Faker

# Initialize Faker
fake = Faker()

def generate_employee_schedules(employee_ids, num_days=30, historical=True):
    """
    Generates schedules for employees over a specified number of days.

    Args:
        employee_ids (list): List of employee IDs.
        num_days (int): Number of days to generate schedules for.
        historical (bool): If True, generates schedules for past dates; 
                           otherwise, generates for future dates.

    Returns:
        pd.DataFrame: A DataFrame containing employee schedules.
    """
    if not employee_ids:
        raise ValueError("Employee IDs must be provided.")

    schedules = []
    schedule_id = 1

    for employee_id in employee_ids:
        for day in range(num_days):
            # Generate a date based on historical or future requirement
            if historical:
                schedule_date = datetime.now() - timedelta(days=day + 1)
            else:
                schedule_date = datetime.now() + timedelta(days=day + 1)
            
            # Randomize shift start and duration
            shift_start = fake.date_time_between_dates(
                datetime_start=schedule_date.replace(hour=6, minute=0, second=0),
                datetime_end=schedule_date.replace(hour=14, minute=0, second=0)
            )
            shift_duration = timedelta(hours=random.randint(4, 8))  # Random shift of 4-8 hours
            shift_end = shift_start + shift_duration

            # Add the schedule entry
            schedules.append({
                "schedule_id": schedule_id,
                "employee_id": employee_id,
                "start_time": shift_start,
                "end_time": shift_end
            })
            schedule_id += 1

    return pd.DataFrame(schedules)




In [6]:
employee_id_list = list(range(1, 91))

In [ ]:
# Example usage
historical_schedules = generate_employee_schedules(employee_id_list, num_days=30, historical=True)
print(historical_schedules)

      schedule_id  employee_id                 start_time  \
0               1            1 2024-12-07 08:24:37.697042   
1               2            1 2024-12-06 09:58:22.047213   
2               3            1 2024-12-05 09:14:43.330791   
3               4            1 2024-12-04 08:36:56.925051   
4               5            1 2024-12-03 06:35:27.849978   
...           ...          ...                        ...   
2695         2696           90 2024-11-12 09:22:44.969006   
2696         2697           90 2024-11-11 13:05:43.362329   
2697         2698           90 2024-11-10 10:52:50.312849   
2698         2699           90 2024-11-09 08:37:41.218811   
2699         2700           90 2024-11-08 10:59:15.722905   

                       end_time  
0    2024-12-07 15:24:37.697042  
1    2024-12-06 17:58:22.047213  
2    2024-12-05 13:14:43.330791  
3    2024-12-04 13:36:56.925051  
4    2024-12-03 11:35:27.849978  
...                         ...  
2695 2024-11-12 15:22:44.9690

##### historical employee schedules

In [41]:
# Define the table and column structure
table_name = "employee_schedule"
columns = ", ".join(historical_schedules.columns)
placeholders = ", ".join(["%s"] * len(historical_schedules.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in historical_schedules.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the employee_schedule table.


#### Employee Time Tracking

In [43]:
import pandas as pd
import random
from datetime import datetime, timedelta
from faker import Faker

# Initialize Faker
fake = Faker()

# Function to generate historical time tracking data
def generate_time_tracking_data(employee_ids, num_days=30):
    """
    Generates historical time tracking data for employees.

    Args:
        employee_ids (list): List of employee IDs.
        num_days (int): Number of past days to generate data for.

    Returns:
        pd.DataFrame: A DataFrame containing employee time tracking data.
    """
    if not employee_ids:
        raise ValueError("Employee IDs must be provided.")

    time_tracking_data = []
    time_tracking_id = 1  # Start tracking ID (if needed for database insertion)

    for employee_id in employee_ids:
        for day in range(num_days):
            # Generate a historical date
            work_date = datetime.now() - timedelta(days=day + 1)
            
            # Randomize shift start and duration
            shift_start = fake.date_time_between_dates(
                datetime_start=work_date.replace(hour=6, minute=0, second=0),
                datetime_end=work_date.replace(hour=14, minute=0, second=0)
            )
            shift_duration = timedelta(hours=random.randint(4, 8))  # Random shift duration of 4-8 hours
            shift_end = shift_start + shift_duration
            
            # Add the time tracking entry
            time_tracking_data.append({
                "time_tracking_id": time_tracking_id,
                "employee_id": employee_id,
                "start_time": shift_start,
                "end_time": shift_end,
                "created_at": datetime.now(),
                "updated_at": datetime.now(),
            })
            time_tracking_id += 1

    return pd.DataFrame(time_tracking_data)



In [44]:

time_tracking_df = generate_time_tracking_data(employee_id_list, num_days=30)

time_tracking_df



,time_tracking_id,employee_id,start_time,end_time,created_at,updated_at
0,1,1,2024-12-07 06:03:25.463102,2024-12-07 12:03:25.463102,2024-12-08 11:44:43.809619,2024-12-08 11:44:43.809620
1,2,1,2024-12-06 07:00:30.033296,2024-12-06 15:00:30.033296,2024-12-08 11:44:43.809737,2024-12-08 11:44:43.809738
2,3,1,2024-12-05 09:06:36.998231,2024-12-05 16:06:36.998231,2024-12-08 11:44:43.809827,2024-12-08 11:44:43.809828
3,4,1,2024-12-04 06:59:10.628482,2024-12-04 11:59:10.628482,2024-12-08 11:44:43.809911,2024-12-08 11:44:43.809912
4,5,1,2024-12-03 13:53:27.992262,2024-12-03 21:53:27.992262,2024-12-08 11:44:43.809993,2024-12-08 11:44:43.809994
...,...,...,...,...,...,...
2695,2696,90,2024-11-12 08:24:54.063371,2024-11-12 13:24:54.063371,2024-12-08 11:44:43.919993,2024-12-08 11:44:43.919994
2696,2697,90,2024-11-11 12:04:01.154397,2024-11-11 16:04:01.154397,2024-12-08 11:44:43.920020,2024-12-08 11:44:43.920020
2697,2698,90,2024-11-10 07:26:50.569933,2024-11-10 13:26:50.569933,2024-12-08 11:44:43.920047,2024-12-08 11:44:43.920047
2698,2699,90,2024-11-09 06:02:55.150144,2024-11-09 11:02:55.150144,2024-12-08 11:44:43.920074,2024-12-08 11:44:43.920074


In [45]:
# Define the table and column structure
table_name = "employee_time_tracking"
columns = ", ".join(time_tracking_df.columns)
placeholders = ", ".join(["%s"] * len(time_tracking_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in time_tracking_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the employee_time_tracking table.


#### Payroll

In [8]:
import pandas as pd
import random
from datetime import datetime, timedelta

def generate_fake_payroll(employee_ids, num_pay_periods=12):
    """
    Generates fake payroll data for employees with consistent hourly wages.

    Args:
        employee_ids (list): List of employee IDs.
        num_pay_periods (int): Number of pay periods to generate for each employee.

    Returns:
        pd.DataFrame: A DataFrame containing payroll data.
    """
    if not employee_ids:
        raise ValueError("Employee IDs must be provided.")

    payroll_data = []
    payroll_id = 1  # Start payroll ID (if needed for database insertion)

    # Assign consistent hourly wage to each employee
    employee_wages = {
        employee_id: round(random.uniform(15.00, 50.00), 2)
        for employee_id in employee_ids
    }

    for employee_id in employee_ids:
        for period in range(num_pay_periods):
            # Generate pay period start and end dates (bi-weekly schedule)
            pay_period_end = datetime.now() - timedelta(weeks=2 * period)
            pay_period_start = pay_period_end - timedelta(days=13)  # Bi-weekly pay period (14 days)

            # Retrieve consistent hourly wage for the employee
            hourly_wage = employee_wages[employee_id]

            # Add payroll entry
            payroll_data.append({
                "payroll_id": payroll_id,
                "employee_id": employee_id,
                "pay_period_start": pay_period_start.date(),
                "pay_period_end": pay_period_end.date(),
                "hour_wage": hourly_wage,
            })
            payroll_id += 1

    return pd.DataFrame(payroll_data)

In [9]:
# Example usage
payroll_df = generate_fake_payroll(employee_id_list, num_pay_periods=2)

payroll_df


,payroll_id,employee_id,pay_period_start,pay_period_end,hour_wage
0,1,1,2024-11-25,2024-12-08,29.16
1,2,1,2024-11-11,2024-11-24,29.16
2,3,2,2024-11-25,2024-12-08,23.00
3,4,2,2024-11-11,2024-11-24,23.00
4,5,3,2024-11-25,2024-12-08,33.66
...,...,...,...,...,...
175,176,88,2024-11-11,2024-11-24,48.21
176,177,89,2024-11-25,2024-12-08,30.63
177,178,89,2024-11-11,2024-11-24,30.63
178,179,90,2024-11-25,2024-12-08,39.36


In [10]:
# Define the table and column structure
table_name = "payroll"
columns = ", ".join(payroll_df.columns)
placeholders = ", ".join(["%s"] * len(payroll_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in payroll_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the payroll table.


#### Vendors Data

In [11]:
import pandas as pd
import random
from faker import Faker

# Initialize Faker
fake = Faker()

def generate_vendors(num_vendors=10):
    """
    Generates vendor data for a grocery store.

    Args:
        num_vendors (int): Number of vendors to generate.

    Returns:
        pd.DataFrame: A DataFrame containing vendor data.
    """
    vendors = []

    for _ in range(num_vendors):
        vendor = {
            "vendor_name": fake.company()[:20],  # Truncate to 20 characters for SQL limit
            "phone_number": f"{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}",
            "email": fake.company_email(),
            "website": fake.url(),
        }
        vendors.append(vendor)

    # Ensure at least one contact method is present (per the SQL constraint)
    for vendor in vendors:
        if random.choice([True, False]):
            vendor["phone_number"] = None
        if not vendor["phone_number"] and random.choice([True, False]):
            vendor["email"] = None
        if not vendor["phone_number"] and not vendor["email"]:
            vendor["website"] = None

    return pd.DataFrame(vendors)

In [23]:
# Generate vendor data
vendors_df = generate_vendors(10)
vendors_df.iloc(1)


In [24]:
vendors_df

,vendor_name,phone_number,email,website
0,"Jones, Johnston and",None,harringtondonald@leonard.com,http://www.palmer-pollard.com/
1,Dunn Ltd,369-582-8782,xdavidson@potter.biz,https://lopez-nunez.org/
2,"Navarro, Ryan and Th",None,cnicholson@young-stone.com,https://www.bradford.com/
3,Townsend-Doyle,669-662-5097,shannon36@carroll.info,https://davis-miller.com/
4,Christensen-Allen,283-735-3010,burketanner@braun-hudson.com,http://www.ruiz.com/
5,"Beck, Henderson and",None,millertimothy@johnson.biz,http://brown-duncan.info/
6,George-Smith,None,edwin39@barrera-herrera.com,http://grimes-davis.biz/
7,Pena-Lamb,409-936-9458,castrojason@sandoval.net,http://www.chapman.com/
8,Garcia LLC,None,laurenhunt@rodriguez-lee.net,https://church.com/
9,Smith-Richardson,203-428-2816,andre65@mcconnell-james.biz,http://rojas-santiago.com/


In [25]:
# Define the table and column structure
table_name = "vendors"
columns = ", ".join(vendors_df.columns)
placeholders = ", ".join(["%s"] * len(vendors_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in vendors_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the vendors table.


#### Products Data

In [ ]:
import pandas as pd
import random

def generate_grocery_products(num_products=100):
    """
    Generates a DataFrame of unique grocery store products.
    
    Args:
        num_products (int): Number of products to generate. Default is 100.
    
    Returns:
        pd.DataFrame: A DataFrame containing product_id, dept_id, vendor_id, brand, and product_name.
    """
    # Departments and example product categories associated with each department
    department_products = {
        1: ["Apples", "Bananas", "Oranges", "Grapes", "Strawberries"],
        2: ["Whole Milk", "Cheddar Cheese", "Greek Yogurt", "Butter", "Cream Cheese"],
        3: ["White Bread", "Baguette", "Croissant", "Muffin", "Donut"],
        4: ["Frozen Pizza", "Ice Cream", "Frozen Vegetables", "Frozen Fries", "Frozen Berries"],
        5: ["Chicken Breast", "Salmon Fillet", "Ground Beef", "Pork Chops", "Shrimp"],
        6: ["Spaghetti Pasta", "Brown Rice", "Canned Beans", "Peanut Butter", "Olive Oil"],
        7: ["Orange Juice", "Bottled Water", "Cola", "Green Tea", "Sparkling Water"],
        8: ["Paper Towels", "Toilet Paper", "Dish Soap", "Laundry Detergent", "Trash Bags"],
        9: ["Shampoo", "Toothpaste", "Deodorant", "Hand Soap", "Body Lotion"],
        10: ["Potato Chips", "Chocolate Bar", "Granola Bars", "Pretzels", "Popcorn"]
    }

    # Example brands
    brands = [
        "FreshFarm", "GoodFood", "OrganicChoice", "BudgetBuy", "NatureDelight",
        "HarvestBest", "DailyEssentials", "GreenValley", "PureTaste", "FamilyFare"
    ]

    # Vendors (1 to 10)
    vendors = list(range(3, 12))

    # Ensure uniqueness by tracking used (brand, product_name) pairs
    used_combinations = set()

    products = []
    product_id = 1

    while len(products) < num_products:
        dept_id = random.choice(list(department_products.keys()))
        vendor_id = random.choice(vendors)
        brand = random.choice(brands)
        product_name = random.choice(department_products[dept_id])

        # Combine brand and product name to form a unique product descriptor
        full_product_name = f"{brand} {product_name}"
        
        # Ensure uniqueness
        if full_product_name not in used_combinations:
            used_combinations.add(full_product_name)
            
            products.append({
                "product_id": product_id,
                "dept_id": dept_id,
                "vendor_id": vendor_id,
                "brand": brand,
                "product_name": full_product_name
            })
            product_id += 1

    return pd.DataFrame(products)

In [34]:
# Example usage:
products_df = generate_grocery_products(100)
products_df


,product_id,dept_id,vendor_id,brand,product_name
0,1,2,3,GreenValley,GreenValley Cream Cheese
1,2,9,9,HarvestBest,HarvestBest Shampoo
2,3,9,10,GreenValley,GreenValley Toothpaste
3,4,9,5,FamilyFare,FamilyFare Body Lotion
4,5,8,4,BudgetBuy,BudgetBuy Paper Towels
...,...,...,...,...,...
95,96,8,3,OrganicChoice,OrganicChoice Paper Towels
96,97,10,4,BudgetBuy,BudgetBuy Popcorn
97,98,10,11,NatureDelight,NatureDelight Granola Bars
98,99,4,4,PureTaste,PureTaste Ice Cream


In [37]:
# Define the table and column structure
table_name = "products"
columns = ", ".join(products_df.columns)
placeholders = ", ".join(["%s"] * len(products_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in products_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the products table.


#### Price History Data

In [97]:
import pandas as pd
import random
from datetime import datetime, timedelta

def generate_price_history_with_timestamps(products_df, start_date="2023-10-01", num_stores=3):
    """
    Generates price history for products from a given start date to the present with timestamps.
    The 'end_date' column has been removed from the output.

    Args:
        products_df (pd.DataFrame): DataFrame containing product data.
        start_date (str): The start date for price history generation (format: YYYY-MM-DD).
        num_stores (int): Number of stores associated with each product.

    Returns:
        pd.DataFrame: A DataFrame containing price history records with timestamps.
    """
    # Convert start_date to a datetime object
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.now()
    price_history = []

    # Iterate through each product
    for _, product in products_df.iterrows():
        product_id = product["product_id"]
        
        # Generate price history for each store
        for store_id in range(1, num_stores + 1):
            current_date = start_date
            while current_date < end_date:
                price = round(random.uniform(1.00, 50.00), 2)  # Random price between $1.00 and $50.00
                
                # Calculate effective_date with timestamps
                effective_date = current_date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59))

                # Append the price history record
                price_history.append({
                    "product_id": product_id,
                    "store_id": store_id,
                    "price": price,
                    "effective_date": effective_date
                })

                # Update the current date for the next price period
                current_date = effective_date + timedelta(days=random.randint(7, 14))  # Price valid for 1-2 weeks

    return pd.DataFrame(price_history)


In [98]:
# Generate price history with timestamps
price_history_df = generate_price_history_with_timestamps(products_df)

In [104]:
price_history_df.tail(10)

,product_id,store_id,price,effective_date
12012,100,3,24.88,2024-08-21 06:28:00
12013,100,3,32.73,2024-08-30 11:02:00
12014,100,3,48.14,2024-09-11 13:30:00
12015,100,3,10.33,2024-09-23 05:32:00
12016,100,3,42.86,2024-10-02 07:25:00
12017,100,3,17.63,2024-10-16 03:27:00
12018,100,3,46.76,2024-10-29 15:36:00
12019,100,3,13.29,2024-11-09 07:42:00
12020,100,3,7.91,2024-11-21 09:37:00
12021,100,3,9.32,2024-12-02 01:34:00


In [103]:
# Define the table and column structure
table_name = "price_history"
columns = ", ".join(price_history_df.columns)
placeholders = ", ".join(["%s"] * len(price_history_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in price_history_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the price_history table.


#### Deliveries/Inventory Data

In [105]:
import pandas as pd
import random
from datetime import datetime, timedelta

def generate_deliveries_data(num_deliveries=1000, vendors=range(3, 13), products=range(1, 101), stores=range(1, 4)):
    """
    Generates data for deliveries to populate the deliveries table.

    Args:
        num_deliveries (int): Number of delivery records to generate.
        vendors (range or list): Range or list of vendor IDs (default: 3 to 12 inclusive).
        products (range or list): Range or list of product IDs (default: 1 to 100 inclusive).
        stores (range or list): Range or list of store IDs (default: 1 to 3 inclusive).

    Returns:
        pd.DataFrame: A DataFrame containing delivery data.
    """
    deliveries = []
    
    for _ in range(num_deliveries):
        vendor_id = random.choice(vendors)
        product_id = random.choice(products)
        store_id = random.choice(stores)
        delivery_date = datetime.now() - timedelta(days=random.randint(1, 180))  # Random date in the past 6 months
        quantity = random.randint(100, 1000)  # Random quantity between 100 and 1000

        deliveries.append({
            "vendor_id": vendor_id,
            "product_id": product_id,
            "store_id": store_id,
            "delivery_date": delivery_date.date(),
            "quantity": quantity
        })
    
    return pd.DataFrame(deliveries)

In [106]:
# Generate delivery data
deliveries_df = generate_deliveries_data(num_deliveries=1000)

# Display the DataFrame
deliveries_df.head(10)

,vendor_id,product_id,store_id,delivery_date,quantity
0,4,89,1,2024-07-29,853
1,11,3,1,2024-08-12,398
2,9,11,3,2024-06-20,1000
3,4,66,3,2024-10-07,935
4,5,100,2,2024-08-14,681
5,5,97,2,2024-08-25,616
6,6,29,3,2024-09-19,263
7,12,94,1,2024-11-04,650
8,6,100,1,2024-08-27,534
9,8,83,2,2024-09-21,876


In [ ]:
# Define the table and column structure
table_name = "deliveries"
columns = ", ".join(deliveries_df.columns)
placeholders = ", ".join(["%s"] * len(deliveries_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in deliveries_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the deliveries table.


#### Sale and Transaction Data

In [119]:
import pandas as pd
import random
from datetime import datetime, timedelta

def generate_transactions_and_sales(num_transactions=5000):
    """
    Generates transactions and sales data starting from October 1, 2023.

    Args:
        num_transactions (int): Number of transactions to generate.

    Returns:
        tuple: Two DataFrames, one for transactions and one for sales.
    """
    transactions = []
    sales = []

    transaction_id = 1
    start_date = datetime(2024, 10, 1)  # Start date for transactions
    end_date = datetime.now()  # Current date as the end date

    for _ in range(num_transactions):
        # Randomize store_id, transaction_type, and transaction_date
        store_id = random.randint(1, 3)
        transaction_type = 0 if random.random() < 0.95 else 1  # 95% sales, 5% returns
        transaction_date = start_date + timedelta(
            days=random.randint(0, (end_date - start_date).days),  # Random day within range
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59)
        )

        # Add transaction record
        transactions.append({
            "transaction_id": transaction_id,
            "store_id": store_id,
            "transaction_date": transaction_date,
            "transaction_type": transaction_type
        })

        # Generate a random number of sales/returns for this transaction
        num_items = random.randint(1, 5)  # Each transaction involves 1 to 5 items
        for _ in range(num_items):
            product_id = random.randint(1, 100)  # Random product ID
            quantity = random.randint(1, 5)  # Random quantity (1 to 10)

            # Add sale record (quantity should be positive, even for returns)
            sales.append({
                "transaction_id": transaction_id,
                "product_id": product_id,
                "quantity": quantity
            })

        transaction_id += 1

    # Convert to DataFrames
    transactions_df = pd.DataFrame(transactions)
    sales_df = pd.DataFrame(sales)

    return transactions_df, sales_df


In [159]:
# Generate transactions and sales data
transactions_df, sales_df = generate_transactions_and_sales(500)

# Display samples
print("Transactions Sample:")
print(transactions_df.head())

print("\nSales Sample:")
print(sales_df.head())

Transactions Sample:
   transaction_id  store_id    transaction_date  transaction_type
0               1         2 2024-10-02 11:35:00                 0
1               2         3 2024-11-20 16:42:00                 0
2               3         1 2024-10-01 20:07:00                 0
3               4         3 2024-10-19 03:54:00                 0
4               5         3 2024-10-18 04:13:00                 0

Sales Sample:
   transaction_id  product_id  quantity
0               1          14         2
1               1          18         2
2               1          65         1
3               1          80         4
4               1          15         4


In [160]:
# Define the table and column structure
table_name = "transactions"
columns = ", ".join(transactions_df.columns)
placeholders = ", ".join(["%s"] * len(transactions_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in transactions_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the transactions table.


In [161]:
# Define the table and column structure
table_name = "sales"
columns = ", ".join(sales_df.columns)
placeholders = ", ".join(["%s"] * len(sales_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in sales_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the sales table.


#### Customer Information

In [166]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

def generate_customers(num_customers=1000):
    """
    Generates customer data for populating the customers table.

    Args:
        num_customers (int): Number of customers to generate.

    Returns:
        pd.DataFrame: A DataFrame containing customer data.
    """
    fake = Faker()
    customers = []

    for _ in range(num_customers):
        first_name = fake.first_name()
        last_name = fake.last_name()
        email = fake.unique.email()
        phone_number = f"{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
        date_joined = datetime.now() - timedelta(days=random.randint(0, 365 * 2))  # Random date in past 2 years

        customers.append({
            "first_name": first_name,
            "last_name": last_name,
            "email": email,
            "phone_number": phone_number,
            "date_joined": date_joined
        })

    return pd.DataFrame(customers)


In [167]:
# Generate customer data
customers_df = generate_customers(1000)

customers_df.head()


,first_name,last_name,email,phone_number,date_joined
0,Angela,Castaneda,heidievans@example.net,868-134-6447,2024-10-13 18:33:10.761905
1,Thomas,Mitchell,rmolina@example.com,459-716-2210,2024-11-03 18:33:10.762654
2,Matthew,Hudson,mcrawford@example.com,711-587-2367,2023-12-29 18:33:10.763098
3,Francisco,Sanders,robin58@example.com,995-362-8665,2023-03-18 18:33:10.763401
4,Shawn,Foster,greglane@example.com,773-221-1731,2023-02-09 18:33:10.763797


In [168]:
# Define the table and column structure
table_name = "customers"
columns = ", ".join(customers_df.columns)
placeholders = ", ".join(["%s"] * len(customers_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in customers_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the customers table.


#### Customer Reviews

##### In customer reviews while leaving a review on the website, customers must be identified so they can be added to mailing list. If a space is left blank it is defaulted to 0 and disregarded in analysis.

##### As part of the ETL process 0 are placeholders in both product and store tables to allow customers to leave reviews naturally about the product, store, or both. 

In [208]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

def generate_customer_reviews(num_reviews=100, customer_ids=range(1, 101), store_ids=range(1, 4), product_ids=range(1, 101)):
    """
    Generates customer reviews for populating the customer_reviews table.

    Args:
        num_reviews (int): Number of reviews to generate.
        customer_ids (range or list): List of customer IDs.
        store_ids (range or list): List of store IDs.
        product_ids (range or list): List of product IDs.

    Returns:
        pd.DataFrame: A DataFrame containing customer review data with store_id and product_id defaulting to 0 if blank.
    """
    fake = Faker()
    reviews = []

    for _ in range(num_reviews):
        customer_id = random.choice(customer_ids)
        store_id = random.choice(store_ids) if random.random() < 0.7 else None  # 70% chance of a store review
        product_id = random.choice(product_ids) if store_id is None or random.random() < 0.7 else None  # 70% chance of a product review
        rating = random.randint(3, 5)  # Random rating between 3 and 5
        review_text = fake.sentence(nb_words=random.randint(5, 20)) if random.random() < 0.8 else ""  # 80% chance of review text
        review_date = datetime.now() - timedelta(days=random.randint(1, 365))  # Random date in past year

        # Ensure at least one of store_id or product_id is not null (to satisfy the CHECK constraint)
        if store_id is None and product_id is None:
            store_id = random.choice(store_ids)

        reviews.append({
            "customer_id": customer_id,
            "store_id": int(store_id) if store_id is not None else 0,  # Default to 0 if None
            "product_id": int(product_id) if product_id is not None else 0,  # Default to 0 if None
            "rating": rating,
            "review_text": review_text,
            "review_date": review_date
        })

    # Create a DataFrame
    reviews_df = pd.DataFrame(reviews)

    return reviews_df

In [217]:
# Generate reviews
customer_reviews_df = generate_customer_reviews(200)
customer_reviews_df.head(10)

,customer_id,store_id,product_id,rating,review_text,review_date
0,23,3,37,3,Include pretty they age thought coach paper ta...,2024-10-04 19:18:52.869137
1,12,0,42,5,Bit set beat every save table main responsibil...,2024-11-26 19:18:52.869233
2,49,0,26,5,Surface high from pull glass coach.,2024-02-20 19:18:52.869251
3,62,0,82,3,,2024-06-04 19:18:52.869255
4,11,0,64,5,Agree traditional cell as my cup since anyone ...,2024-07-15 19:18:52.869270
5,36,2,99,4,Chance parent cause describe service tree sout...,2024-01-21 19:18:52.869286
6,53,1,0,5,Beat wind reflect baby around wait how add tru...,2024-11-22 19:18:52.869301
7,61,0,4,3,Leg before through strong raise popular try do...,2024-06-27 19:18:52.869315
8,70,0,16,4,Because push several method you through office...,2024-03-03 19:18:52.869329
9,99,0,63,5,Business community arm.,2024-09-08 19:18:52.869343


In [223]:
# Define the table and column structure
table_name = "customer_reviews"
columns = ", ".join(customer_reviews_df.columns)
placeholders = ", ".join(["%s"] * len(customer_reviews_df.columns))  # Placeholder for each column

# SQL insert query
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert each row into the database
for row in customer_reviews_df.itertuples(index=False, name=None):
    cur.execute(insert_query, row)

# Commit the transaction
conn.commit()

print(f"Data successfully inserted into the {table_name} table.")

Data successfully inserted into the customer_reviews table.
